<a href="https://colab.research.google.com/github/rim-yu/procam-calibration/blob/master/procam_calibration_200828.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# You need to mount your google drive  to the /content/gdrive folder of your virtual computer
# located in the colab server

from google.colab import drive
drive.mount("/content/gdrive")
#drive.mount("/content/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

%xmode Verbose
%pdb on

Exception reporting mode: Verbose
Automatic pdb calling has been turned ON


In [3]:
import os
import os.path
from pathlib import Path
#cf: https://treyhunner.com/2018/12/why-you-should-be-using-pathlib/
import glob
import argparse
import cv2
import numpy as np
import json
from matplotlib import pyplot as plt 

In [4]:
def main(proj_height, proj_width, chess_rows, chess_cols, chess_block_size,
    graycode_step, black_thr, white_thr,  camera):
  
# 2160 - 3840 - 10 - 7 - 68 - 1 - 40 - 5 - ""

    proj_shape = (proj_height, proj_width) # (2160, 3840)
    chess_shape = (chess_rows, chess_cols) # (10, 7)
    chess_block_size = chess_block_size # chess_block_size = 68 mm
    gc_step = graycode_step # gc_step = 1
    black_thr = black_thr # 40
    white_thr = white_thr # 5
    camera_param_file = camera # ""

    dirnames = sorted(glob.glob('./capture_*'))
    if len(dirnames) == 0: # capture_* 형식으로 된 폴더가 없을 경우 error 뱉음. 
        print('Directories \'./capture_*\' were not found')
        return

    print('Searching input files ...') # capture_* 형식으로 된 폴더가 있을 경우 파일들을 찾음. 
    used_dirnames = [] 
    gc_fname_lists = [] # 파일 이름 리스트. 
    for dname in dirnames: # capture_* 폴더 돌림. * = 1->4
        #gc_fnames = sorted(glob.glob(dname + '/graycode_*')) # graycode_* 파일들 가져옴. 
        gc_fnames = sorted(glob.glob(dname + '/graycode*'))
        if len(gc_fnames) == 0:  
            continue
        used_dirnames.append(dname) # capture_* 폴더가 순차적으로 붙음. 
        gc_fname_lists.append(gc_fnames) # 파일 이름 리스트. graycode_*가 붙음. * = 00->49
        print(' \'' + dname + '\' was found') # 폴더 찾았당. 

    camP = None # ?
    cam_dist = None # ? 
    path, ext = os.path.splitext(camera_param_file) # camera_param_file = "", 파일 이름을 확장자로부터 쪼개줌. 
    if(ext == ".json"): # 확장자명. 
        camP, cam_dist = loadCameraParam(camera_param_file) # 카메라 파라미터 저장된 json 파일 가져옴. 
        print('load camera parameters')
        print(camP)
        print(cam_dist)

    calibrate(used_dirnames, gc_fname_lists,
              proj_shape, chess_shape, chess_block_size, gc_step, black_thr, white_thr,
               camP, cam_dist) 
    # calibrate 함수 쓰는데, capture_* 폴더명들, graycode_* 파일명들, 그외 각종 변수들 + camP, cam_dist 

In [6]:
def printNumpyWithIndent(tar, indentchar):
    print(indentchar + str(tar).replace('\n', '\n' + indentchar))  

In [5]:
def loadCameraParam(json_file):
    with open(json_file, 'r') as f:
        param_data = json.load(f)
        P = param_data['camera']['P']
        d = param_data['camera']['distortion']
        return np.array(P).reshape([3,3]), np.array(d) # ok 

In [7]:
def calibrate(dirnames, gc_fname_lists, proj_shape, chess_shape, chess_block_size, gc_step, black_thr, white_thr, camP, camD):
# proj_shape = (2160, 3840) chess_shape = (10, 7)    

    # (1) object points
    objps = np.zeros((chess_shape[0]*chess_shape[1], 3), np.float32) 
    # object points. 70행 3열 2차원 배열이 나타남. 
    objps[:, :2] = chess_block_size * \
        np.mgrid[0:chess_shape[0], 0:chess_shape[1]].T.reshape(-1, 2)
        # [[0,0], ..., [9,0], ..., [9,9]] 이렇게 바뀜. 
        # 모든 행 2열 전까지. 즉 열 2개만. 3열은 0만.
        # ok

    # (2) projector size -> gray code 
    print('Calibrating ...') # 계산중임다. 
    gc_height = int((proj_shape[0]-1)/gc_step)+1 # 2160. 
    gc_width = int((proj_shape[1]-1)/gc_step)+1 # 3840. 

##### 이 부분이 
    graycode = cv2.structured_light_GrayCodePattern.create(
        gc_width, gc_height) # 3840 * 2160 -> 이거로 50장 만들어내는 듯.
    graycode.setBlackThreshold(black_thr)  
    graycode.setWhiteThreshold(white_thr) 
##### 이해가 잘 안가요.  

    # (3) graycode_* size, patch_size 
    cam_shape = cv2.imread(gc_fname_lists[0][0], cv2.IMREAD_GRAYSCALE).shape 
    # cam_shape = cv2.imread(gc_fname_lists[0][0]).shape
    # 이미지 읽기. gc_fname_lists[0][0] = graycode_00
    # gc_fname_list 보니 맨 첫 원소는 [capture_0, graycode_00]임. 
    # cam_shape = 1600*2400 
    patch_size_half = int(np.ceil(cam_shape[1] / 180)) # np.ceil 하면 그 값들이 '올림'된다. cam_shape = (1600, 2400)임. 왜 이건지 모르겠다. 14
    # patch_size_half = int(np.ceil(cam_shape[1] / 270)) # 이렇게 하면 RMS가 5로 떨어지긴 하는데 카메라 내부 파라미터는 완전히 달라서 의미가 없을 듯.   
    print('  patch size :', patch_size_half * 2 + 1) # 29

    cam_corners_list = [] # 카메라 코너들을 모을 예정. 
    cam_objps_list = [] # 카메라 object들을 모을 예정. 
    cam_corners_list2 = [] # 보류
    _cam_corners_list = [] 

    proj_corners_list = [] # 프로젝터 코너들을 모을 예정
    proj_objps_list = [] # 프로젝터 object들을 모을 예정 
    
    # (4) dirnames = capture_*, gc_fname_lists = graycode_*
    for dname, gc_filenames in zip(dirnames, gc_fname_lists): # 폴더-파일  
        print('  checking \'' + dname + '\'') # dirname = capture_*
        if len(gc_filenames) != graycode.getNumberOfPatternImages() + 2:
            print('Error : invalid number of images in \'' + dname + '\'')
            return None 
            # graycode.getNumberOfPatternImages() = 48. 2를 더해서 50이 아닐 경우 에러내라. 

        imgs = []
        _imgpoints = []

        for fname in gc_filenames: # 각 graycode_*
            img = cv2.imread(fname)
            # img = cv2.imread(fname, cv2.IMREAD_GRAYSCALE) # graycode_*를 흑백 모드에서 이미지 로드. 채널 하나구나.       
            # 앞에 _를 붙힌 변수는 image point들을 camera calibration과 동일하게 설정하려 임의로 추가한 코드임다. 
            # _gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            _gray = cv2.imread(fname, cv2.IMREAD_GRAYSCALE)

            if cam_shape != _gray.shape: # (원래) img.shape: # cam_shape = (1600, 2400, 3)
                print('Error : image size of \'' + fname + '\' is mismatch')
                return None # 흑백 모드에서 로드한 이미지가 (1600, 2400)의 사이즈를 갖지 않으면 에러내라.
                # gray scale로 하면 shape이 바뀜.  

            #imgs.append(img) # 빈 리스트에 img(graycode_*)를 연쇄적으로 추가함.  
            imgs.append(_gray)

        black_img = imgs.pop() #imgs.pop() # graycode_49를 black_img로 지정함. 대다수의 0을 가진 매트릭스. graycode_49는 프로젝터에서 검정색 화면만 쏜 것임. 
        white_img = imgs.pop() #imgs.pop() # graycode_48을 white_img로 지정함. 37~60을 가진 매트릭스. graycode_48은 프로젝터에서 하얀색 화면만 쏜 것임.
        # pop() 2회를 통해 imgs에는 48장의 이미지만이 남음.  
        # cv2_imshow(white_img)를 통해 black_img와 white_img를 확인할 수 있다. 

        # 원래 코드 
        res, cam_corners = cv2.findChessboardCorners(white_img, chess_shape) # white_img를 통해 체스보드판의 코너들을 찾아낸다. 
        
        if not res: # res는 코너점들을 찾았는지 알려줌. 
            print('Error : chessboard was not found in \'' +
                  gc_filenames[-2] + '\'')  
            return None

        # 수정한 코드 
        # _ret, _corners = cv2.findChessboardCorners(_gray, (10, 7), None)
        _ret, _corners = cv2.findChessboardCorners(white_img, (10, 7), None)

        if _ret == True:
            _corners2 = cv2.cornerSubPix(white_img, _corners, (11, 11), (-1, -1), (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
            _imgpoints.append(_corners2)

        #_ChessboardCorners = cv2.drawChessboardCorners(img, (10, 7), _corners2, _ret) 
        #if dname == "./capture_00" :
        #    plt.figure(figsize=(30, 30))
        #    plt.imshow(_ChessboardCorners)
        #    plt.axis("off")
        #    plt.show()

        cam_objps_list.append(objps) # 카메라 object들을 모아놓음. 
        cam_corners_list.append(cam_corners) # 카메라 코너들을 모아놓음. 
        _cam_corners_list.append(_corners2) 

        # procam calibration을 본격화하려는 모양. 
        # 위에서 정의한 proj_corners_list = [], proj_objps_list = [] 이 두 변수와 아래 변수들이 뭐가 다른지 생각 좀 해야할 듯. 
        proj_objps = []
        proj_corners = []
        cam_corners2 = []

        # viz_proj_points = np.zeros(proj_shape, np.uint8)
        for corner, objp in zip(cam_corners, objps): # 코너들 - object들   
            c_x = int(round(corner[0][0])) # round 함수는 반올림. c_x는 코너(1).
            c_y = int(round(corner[0][1])) # c_y는 코너(2).
            src_points = []
            dst_points = []
            for dx in range(-patch_size_half, patch_size_half + 1): # -14~15. 29만큼, path_size만큼 for문이 돌아감. 
                for dy in range(-patch_size_half, patch_size_half + 1):
                    x = c_x + dx 
                    y = c_y + dy
                    # 최소 196개의 (x,y) 좌표를 가져와야 ok. 

                    # white_img[y, x]는 (y, x) 지점의 픽셀값을 보여줌. 
                    if int(white_img[y, x]) - int(black_img[y, x]) <= black_thr: # white_img, black_img의 (x, y) 지점에서 차이를 구할 때 그 픽셀값이 black_thr = 40보다 작으면 okay. 
                        continue # 그 차이가 40보다 작아야지 (x, y) 좌표를 가져올 수 있는 건데 capture_1의 경우 그 차이가 40보다 큰 것임. 

                    err, proj_pix = graycode.getProjPixel(imgs, x, y) # projector 관련.

                    if not err:
                        src_points.append((x, y)) # src_points에 (x, y) 값들이 기록되는 것임.   
                        dst_points.append(gc_step*np.array(proj_pix)) # 보류. 
            if len(src_points) < patch_size_half**2: # path_size_half**2 = 196. (x, y)가 모여서 196개는 나와야 하는데 (x, y)가 덜 모이면 에러가 발생하는 것임.  
                print(
                    '    Warning : corner', c_x, c_y,
                    'was skiped because decoded pixels were too few (check your images and threasholds)') # 디코드된 값이 너무 적다고.
                continue
            h_mat, inliers = cv2.findHomography(
                np.array(src_points), np.array(dst_points))
            point = h_mat@np.array([corner[0][0], corner[0][1], 1]).transpose()
            point_pix = point[0:2]/point[2]
            proj_objps.append(objp)
            proj_corners.append([point_pix])
            cam_corners2.append(corner)
            # viz_proj_points[int(round(point_pix[1])),
            #                 int(round(point_pix[0]))] = 255

        if len(proj_corners) < 3:
            print('Error : too few corners were found in \'' +
                  dname + '\' (less than 3)')
            return None
        proj_objps_list.append(np.float32(proj_objps))
        proj_corners_list.append(np.float32(proj_corners))
        cam_corners_list2.append(np.float32(cam_corners2))
        # cv2.imwrite('visualize_corners_projector_' +
        #             str(cnt) + '.png', viz_proj_points)
        # cnt += 1

    print('Initial solution of camera\'s intrinsic parameters')
    cam_rvecs = []
    cam_tvecs = []
    if(camP is None):
        ret, cam_int, cam_dist, cam_rvecs, cam_tvecs = cv2.calibrateCamera(
            cam_objps_list, _cam_corners_list, _gray.shape[::-1], None, None, None, None) # cam_shape -> _gray.shape[::-1]
        print('  RMS :', ret)
    else:
        for objp, corners in zip(cam_objps_list, _cam_corners_list):##
            ret, cam_rvec, cam_tvec = cv2.solvePnP(objp, corners, camP, camD) # 외부 파라미터 구하는 함수.  
            cam_rvecs.append(cam_rvec)
            cam_tvecs.append(cam_tvec)
            print('  RMS :', ret)
        cam_int = camP
        cam_dist = camD
    print('  Intrinsic parameters :')
    printNumpyWithIndent(cam_int, '    ')
    print('  Distortion parameters :')
    printNumpyWithIndent(cam_dist, '    ')
    cam_rotation_matrix = np.zeros(shape=(3, 3))
    cv2.Rodrigues(cam_rvecs[0], cam_rotation_matrix) 
    # Rodrigues로 표현된(3*1) R vectors를 3*3으로 표현하기 위함.
    # 체스보드를 바닥에 딱 붙힌 capture_00에 대해서 rotation vector를 구하겠음.  
    print('  cam_rvecs_capture_00 :')
    printNumpyWithIndent(cam_rotation_matrix, '    ')
    print('  cam_tvecs_capture_00 :')
    printNumpyWithIndent(cam_tvecs[0], '    ')
    print()

    print('Initial solution of projector\'s parameters')
    ret, proj_int, proj_dist, proj_rvecs, proj_tvecs = cv2.calibrateCamera(
        proj_objps_list, proj_corners_list, proj_shape, None, None, None, None)
    
    print('  RMS :', ret)
    print('  Intrinsic parameters :')
    printNumpyWithIndent(proj_int, '    ')
    print('  Distortion parameters :')
    printNumpyWithIndent(proj_dist, '    ')
    proj_rotation_matrix = np.zeros(shape=(3, 3))
    cv2.Rodrigues(proj_rvecs[0], proj_rotation_matrix) # Rodrigues로 표현된(3*1) R vectors를 3*3으로 표현하기 위함. 
    print('  proj_rvecs_capture_00 :')
    printNumpyWithIndent(proj_rotation_matrix, '    ')
    print('  proj_tvecs_capture_00 :')
    printNumpyWithIndent(proj_tvecs[0], '    ') 
    print()

    print('=== Result ===')
    ret, cam_int, cam_dist, proj_int, proj_dist, cam_proj_rmat, cam_proj_tvec, E, F = cv2.stereoCalibrate(
        proj_objps_list, cam_corners_list2, proj_corners_list, cam_int, cam_dist, proj_int, proj_dist, None)
    print('  RMS :', ret)
    print('  Camera intrinsic parameters :')
    printNumpyWithIndent(cam_int, '    ')
    print('  Camera distortion parameters :')
    printNumpyWithIndent(cam_dist, '    ')
    print('  Projector intrinsic parameters :')
    printNumpyWithIndent(proj_int, '    ')
    print('  Projector distortion parameters :')
    printNumpyWithIndent(proj_dist, '    ')
    print('  Rotation matrix / translation vector from camera to projector')
    print('  (they translate points from camera coord to projector coord) :')
    printNumpyWithIndent(cam_proj_rmat, '    ')
    printNumpyWithIndent(cam_proj_tvec, '    ')
    print()

    fs = cv2.FileStorage('calibration_result.xml', cv2.FILE_STORAGE_WRITE)
    fs.write('img_shape', cam_shape)
    fs.write('rms', ret)
    fs.write('cam_int', cam_int)
    fs.write('cam_dist', cam_dist)
    fs.write('proj_int', proj_int)
    fs.write('proj_dist', proj_dist)
    fs.write('roration', cam_proj_rmat)
    fs.write('translation', cam_proj_tvec)
    fs.release()

In [20]:
path = Path('/content/gdrive/My Drive/danbi-project/data/200805_1')
#%cd /content/gdrive/My\ Drive/procam-calibration/sample_data
# !pwd # show me the current working directory

In [21]:
cd /content/gdrive/"My Drive"/danbi-project/data/200805_1

/content/gdrive/My Drive/danbi-project/data/200805_1


In [22]:
ls

-/                      capture_01/  capture_04/  capture_08/  capture_12/
calibration_result.xml  capture_02/  capture_06/  capture_10/  capture_14/
capture_00/             capture_03/  capture_07/  capture_11/  capture_15/


In [23]:
proj_height = 2160 
proj_width = 3840
chess_rows = 10 
chess_cols = 7  
chess_block_size = 96
graycode_step = 1
black_thr = 40
white_thr = 2
camera =  ""

main(proj_height, proj_width, chess_rows, chess_cols, chess_block_size,
    graycode_step, black_thr, white_thr,  camera)

Searching input files ...
 './capture_00' was found
 './capture_01' was found
 './capture_02' was found
 './capture_03' was found
 './capture_04' was found
 './capture_06' was found
 './capture_07' was found
 './capture_08' was found
 './capture_10' was found
 './capture_11' was found
 './capture_12' was found
 './capture_14' was found
 './capture_15' was found
Calibrating ...
  patch size : 29
  checking './capture_00'
  checking './capture_01'
  checking './capture_02'
  checking './capture_03'
  checking './capture_04'
  checking './capture_06'
  checking './capture_07'
  checking './capture_08'
  checking './capture_10'
  checking './capture_11'
  checking './capture_12'
  checking './capture_14'
  checking './capture_15'
Initial solution of camera's intrinsic parameters
  RMS : 0.7241568277087241
  Intrinsic parameters :
    [[2.02303721e+03 0.00000000e+00 1.20980420e+03]
     [0.00000000e+00 2.02418275e+03 8.21426626e+02]
     [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
  Dis

In [ ]:
####
import numpy as np

tvec_wc = np.array([276.39040931, 204.66910348, 1732.75087476])
rvec_wc = np.array([[-0.99601225, -0.08855388, 0.01085398],
                   [0.08866735, -0.99600637, 0.01046007],
                   [0.00988435, 0.01138075, 0.99988638]])
print("tvec_wc : \n", tvec_wc)
print("rvec_wc : \n", rvec_wc)
# Pc = R*Pw + T
# Pw = R.T*Pc - R.T*T

tvec_cw = -np.matmul(rvec_wc.T, tvec_wc)
rvec_cw = rvec_wc.T
print("tvec_cw : \n", tvec_cw)
print("rvec_cw : \n", rvec_cw)

tvec_cp = np.array([368.10657134l, 216.21286606, 377.9167948])
rvec_cp = np.array([[0.99663129, -0.08155148, -0.00868462],
                   [0.08135873, 0.99647157, -0.0206198],
                   [0.01033555, 0.01984377, 0.99974967]])
print("tvec_cp : \n", tvec_cp)
print("rvec_cp : \n", rvec_cp)

tvec_pc = -np.matmul(rvec_cp.T, tvec_cp)
rvec_pc = rvec_cp.T
print("tvec_pc : \n", tvec_pc)
print("rvec_pc : \n", rvec_pc)

tvec_wp = np.array([611.82781084, 407.56942583, 2119.22585348])
rvec_wp = np.array([[-9.99984325e-01, -5.59901665e-03, 5.84897872e-06],
                   [5.59862145e-03, -9.99925110e-01, -1.08825467e-02],
                   [6.67801009e-05 -1.08823434e-02  9.99940783e-01]]
print("tvec_wp : \n", tvec_wp)
print("rvec_wp : \n", rvec_wp)

tvec_pw = -np.matmul(rvec_wp.T, tvec_wp)
rvec_pw = rvec_wp.T
print("tvec_pw : \n", tvec_pw)
print("rvec_pw : \n", rvec_pw)